In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
sys.path.insert(1, "/shared/opencv/release/lib/python2.7/dist-packages/")
import cv2
import random
from collections import defaultdict

from sklearn.cluster import KMeans
import pandas as pd

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from sift_spm_utilities import *
from learning_utilities import *
from annotation_utilities import *

Setting environment for AWS compute node


No vtk
Cannot import mxnet.


In [2]:
# CSHL_SPM_ROOTDIR = '/shared/CSHL_SIFT_SPM_features/'
M = 200 # vocabulary size
colors = np.vstack([(0,0,0), np.random.randint(0, 255, (M, 3))])

In [3]:
win_id = 1

In [4]:
stack = 'MD589'

# Build Vocabulary

In [5]:
sift = cv2.xfeatures2d.SIFT_create()

In [31]:
for stack in ['MD589']:
    for sec in metadata_cache['valid_sections'][stack]:
        
        image = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2, version='gray')
        mask = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2, version='mask', resol='thumbnail')
        xmin_tb, xmax_tb, ymin_tb, ymax_tb = bbox_2d(mask)
        xmin = xmin_tb * 32
        xmax = (xmax_tb + 1) * 32
        ymin = ymin_tb * 32
        ymax = (ymax_tb + 1) * 32

        img = image[ymin:ymax+1, xmin:xmax+1].copy()

        t = time.time()
        keypoints, descriptors = sift.detectAndCompute(img, None);
        keypoints = np.array([k.pt for k in keypoints]) + (xmin, ymin)
        
        fp = DataManager.get_sift_keypoints_filepath(stack=stack, section=sec)
        create_parent_dir_if_not_exists(fp)
        bp.pack_ndarray_file(keypoints, fp)
        upload_to_s3(fp)
        
        fp = DataManager.get_sift_descriptors_filepath(stack=stack, section=sec)
        create_parent_dir_if_not_exists(fp)
        bp.pack_ndarray_file(descriptors, fp)
        upload_to_s3(fp)
        
        sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) # 60s

KeyboardInterrupt: 

In [28]:
# viz = gray2rgb(img)
# for x, y in keypoints:
#     cv2.circle(viz, (int(x), int(y)), 5, (255,0,0))

In [30]:
# display_image(viz[1000:5000, 1000:5000])

/shared/MouseBrainAtlas/spm/tmp.jpg

# or use author's own binary, 10x slower.

In [42]:
from skimage.io import load_sift

In [ ]:
import uuid

for stack in ['MD589']:
    for sec in metadata_cache['valid_sections'][stack][:1]:
        
        image = DataManager.load_image_v2(stack=stack, section=134, prep_id=2, version='gray')
        mask = DataManager.load_image_v2(stack=stack, section=134, prep_id=2, version='mask', resol='thumbnail')
        xmin_tb, xmax_tb, ymin_tb, ymax_tb = bbox_2d(mask)
        xmin = xmin_tb * 32
        xmax = (xmax_tb + 1) * 32
        ymin = ymin_tb * 32
        ymax = (ymax_tb + 1) * 32

        img = image[ymin:ymax+1, xmin:xmax+1].copy()

        
        keypoints_allBlocks = []
        descriptors_allBlocks = []
        
        # the binary cannot deal with large data
        for y in range(0, img.shape[0], 3000):
            for x in range(0, img.shape[1], 3000):
                t = time.time()
                
                uid = str(uuid.uuid1()).split('-')[0]
                in_fp = '/tmp/%s.pgm' % uid
                imsave(in_fp, img[y:y+3000, x:x+3000])
                out_fp = '/tmp/%s.key' % uid
                execute_command('/shared/siftDemoV4/sift <%s >%s' % (in_fp, out_fp))

                sift_data = load_sift(out_fp)
                descriptors = sift_data['data'].shape
                keypoints = np.c_[sift_data['column'], sift_data['row']]
                print len(keypoints), 'keypoints'
                # sift_data['orientation']
                # sift_data['scale']
                
                keypoints_allBlocks.append(keypoints)
                descriptors_allBlocks.append(descriptors)

        #         keypoints, descriptors = sift.detectAndCompute(img, None);
                sys.stderr.write('sift: %.2f seconds\n' % (time.time() - t)) 
            
        keypoints = np.concatenate(keypoints_allBlocks)
        descriptors = np.concatenate(descriptors_allBlocks)

In [23]:
indices_map = defaultdict(list)
hists0_allLandmarks = defaultdict(list)
hists1_allLandmarks = defaultdict(list)
hists2_allLandmarks = defaultdict(list)

# for stack in ['MD589', 'MD594', 'MD585']:
for stack in ['MD589']:

#     username = 'yuncong'
#     label_polygons = load_label_polygons_if_exists(stack, username, force=False, 
#                                                    annotation_rootdir=annotation_midbrainIncluded_rootdir)

    grid_indices_perSec_perStruct = DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=True)
    
#     image_width, image_height = DataManager.get_image_dimension(stack)
#     grid_spec = (patch_size, stride, image_width, image_height)
#     sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)

    patch_size = windowing_settings[win_id]['patch_size']
    half_size = patch_size / 2
    stride = windowing_settings[win_id]['spacing']
    image_width, image_height = metadata_cache['image_shape'][stack]
    grid_spec = (patch_size, stride, image_width, image_height)

    sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)
    
    for sec in metadata_cache['valid_sections'][stack][:1]:
        print sec

        labelmap = compute_labelmap(stack, sec)
        
        mask_tb = DataManager.load_image_v2(stack, prep_id=2, section=sec, resol='thumbnail', version='mask')
        
        for name, indices in grid_indices_perSec_perStruct.iloc[sec].dropna().iteritems():
            if name == 'bg' or name == 'noclass':
                continue
                
            print name
            num_sample = 20
            indices_sampled = np.random.choice(indices, min(len(indices), num_sample), replace=False)
            sample_locs = sample_locations[indices_sampled]

            hists_arr0, hists_arr1, hists_arr2 = compute_spm_histograms(labelmap, sample_locs, patch_size=patch_size, M=M)

            hists0_allLandmarks[name].append(hists_arr0)
            hists1_allLandmarks[name].append(hists_arr1)
            hists2_allLandmarks[name].append(hists_arr2)

            indices_map[name].append(indices_sampled)

hists0_allLandmarks.default_factory = None
hists1_allLandmarks.default_factory = None
hists2_allLandmarks.default_factory = None

indices_map.default_factory = None

92


[Errno 13] Permission denied: '/oasis'


AttributeError: 'module' object has no attribute 'SIFT'

In [ ]:
hists0_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists0_allLandmarks.iteritems()}
hists1_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists1_allLandmarks.iteritems()}
hists2_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists2_allLandmarks.iteritems()}
indices_map = {name: [(i,) + l for i, l in enumerate(chain(*indices))] for name, indices in indices_map.iteritems()}